In [1]:
""" LED-LED Classifier
"""

' LED Classifier Analysis\n'

In [3]:
import sys
import os
import random
#sys.path.append('B:/code/cshearer/py/')
#sys.path.append('../cshearer/py/')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import cs_basefunctions as cms
from scipy.stats import zscore,ttest_1samp,sem
#import inference_task as inf
from matplotlib.lines import Line2D
import scipy.stats
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import LeaveOneOut, LeavePOut
import sklearn.metrics as sm
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold, permutation_test_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import statannot
from scipy.stats import bootstrap
import statsmodels.api as sm
from helper_functions import *
sample_rate, ca_sample_rate = cms.sample_rate, cms.ca_sample_rate

# load layer coordinates
layer_split_df = pd.read_csv('layer_x_coordinates.csv')


In [4]:
import statsmodels.api as sm

# Learning Days

In [5]:
# recording days for each mouse
al_99 = ['210128','210129','210130','210131','210201','210202']
al_100 = ['210301','210302','210303','210305','210306']
al_108 = ['210505','210508','210509','210510']
al_124 = ['221017','221018','221019','221020','221021']

rec_days = {'mhb99': al_99,
            'mhb100': al_100,
            'mhb108': al_108,
            'mhb124': al_124}

# recording day relative to start of task
mouse_rec_days = {'mhb99':  np.arange(0,6).tolist(),
                  'mhb100': np.arange(0,3).tolist() + np.arange(4,6).tolist(),
                  'mhb108': [0] + np.arange(3,6).tolist(),
                  'mhb124': np.arange(5).tolist()}

mouse_ids = ['mhb99','mhb100','mhb108', 'mhb124']
correct_choices = [True] # correct trials
sess_type = 'al' # observational learning trials

In [10]:
res = {'mouse':[], 'day':[],'accs':[],'acc_type':[],'layer':[]}

for rep in range(20):
    print('################# rep: ', rep, '  ##############')
    for mouse_id in mouse_ids:

        for rr, rec_day in enumerate(rec_days[mouse_id]):

            print(mouse_id , '  ', rec_day)
            cue_pulse_names = ['ledsucrose','ledwater']
            sess_type = 'al'
            cue_pops = get_layer_cuepop(mouse_id, rec_day, cue_pulse_names, sess_type, correct_trials= False)

            cue_pulse_names = ['toneSucrose','toneWater']
            cue_pop_tts = get_layer_cuepop(mouse_id, rec_day, cue_pulse_names, sess_type, correct_trials= False)

            if cue_pops == None:
                print('empty cue_pop')
                continue
                
            select_cells = np.min([cue_pops[0]['ledsucrose'].shape[1], cue_pops[1]['ledsucrose'].shape[1]])
            for layer, layer_name in enumerate(['L23', 'L56']):

                cue_pop = cue_pops[layer]
                cue_pop_tt = cue_pop_tts[layer]
                              
                if cue_pop['ledsucrose'].shape[1] != select_cells:

                    select_idx = random.sample(range(cue_pop['ledsucrose'].shape[1]),select_cells)
                    select_idx.sort()
                    for cue in cue_pop:
                        cue_pop[cue] = cue_pop[cue][:, select_idx]
                    for cue in cue_pop_tt:
                        cue_pop_tt[cue] = cue_pop_tt[cue][:, select_idx]
                        
                X, Y, Y_string = extract_XY(cue_pop)
                X_tt, Y_tt, Y_string_tt = extract_XY(cue_pop_tt)

                unique, counts = np.unique(Y, return_counts = True)
                if (len(unique)<2) or (min(counts)<2) or (X.shape[1] < 1):
                    do_clf=0
                    print('skipped : ', mouse_id, '  ', rec_day)
                else:
                    do_clf=1

                if do_clf:

                    cv = RepeatedStratifiedKFold(n_splits=np.min(counts),n_repeats=10)
                    scores = np.empty((0,2))
                    for i, (train_index, test_index) in enumerate(cv.split(X,Y)):
                        X_test, X_train, Y_test, Y_train  = X[test_index], X[train_index], Y[test_index], Y[train_index]
                        X_test_tt, X_train_tt, Y_test_tt, Y_train_tt  = X_tt[test_index], X_tt[train_index], Y_tt[test_index], Y_tt[train_index]

                        # scale and transform
                        scaling = StandardScaler()
                        scaling.fit(X_train)

                        # Transform training and test
                        X_test_scaled = scaling.transform(X_test)
                        X_test_tt_scaled = scaling.transform(X_test_tt)

                        model =  LogisticRegression(penalty='l1',solver='liblinear',random_state=0).fit(X_train,Y_train)
                        score1 = balanced_accuracy_score(Y_test_tt, model.predict(X_test_tt_scaled), adjusted=True)
                        score2 = balanced_accuracy_score(Y_test, model.predict(X_test_scaled), adjusted=True)
                        scores = np.vstack([scores, np.array([score1, score2])])
                    print(scores.mean(axis = 0))
                    res['accs'] += scores.mean(axis = 0).tolist()
                    res['acc_type'] += ['tone', 'led']
                    res['mouse']+= [mouse_id]*2
                    res['day']+= [mouse_rec_days[mouse_id][rr]]*2
                    res['layer']+= [layer_name]*2

df_led_al= pd.DataFrame(res)

al_df = df_led_al.groupby(['mouse', 'day', 'acc_type','layer']).agg({'accs':'mean'}).reset_index()
tone_al = al_df[al_df.acc_type == 'tone']
led_al = al_df[al_df.acc_type == 'led']

################# rep:  0   ##############
mhb99    210128
[-0.125  0.525]
[-0.31666667 -0.05833333]
mhb99    210129
[-0.04166667  0.46666667]
[0.35833333 0.28333333]
mhb99    210130
[-0.075       0.56666667]
[0.24166667 0.65833333]
mhb99    210131
[0.1  0.25]
[-0.075  0.25 ]
mhb99    210201
[-0.18333333  0.13333333]
[0.13333333 0.2       ]
mhb99    210202
[ 0.39166667 -0.11666667]
[-0.28333333  0.25      ]
mhb100    210301
[0.23333333 0.28333333]
[0.11666667 0.11666667]
mhb100    210302
[ 0.125 -0.05 ]
[0.01666667 0.28333333]
mhb100    210303
[0.35833333 0.23333333]
[0.3        0.41666667]
mhb100    210305
[-0.11538462  0.43076923]
[-0.20769231  0.63076923]
mhb100    210306
[0.01363636 0.25909091]
[-0.03636364  0.19545455]
mhb108    210505
[-0.08333333 -0.23333333]
[-0.36666667  0.61666667]
mhb108    210508
[0.02307692 0.54615385]
[-0.47692308  0.46923077]
mhb108    210509
[0.  0.5]
[0.         0.40833333]
mhb108    210510
[-0.01666667  0.50833333]
[0.11666667 0.51666667]
mhb124    22

[0.03333333 0.025     ]
[0.025      0.23333333]
mhb124    221019
segment length longer than pulse, length difference:  1
segment length longer than pulse, length difference:  1
[0.         0.30833333]
[0.05833333 0.275     ]
mhb124    221020
[0.53333333 0.225     ]
[0.29166667 0.20833333]
mhb124    221021
[0.075      0.53333333]
[-0.26666667  0.23333333]
################# rep:  6   ##############
mhb99    210128
[-0.08333333  0.475     ]
[-0.3        -0.03333333]
mhb99    210129
[-0.08333333  0.00833333]
[0.3        0.30833333]
mhb99    210130
[-0.08333333  0.65      ]
[0.275      0.65833333]
mhb99    210131
[-0.03333333  0.48333333]
[-0.05833333  0.26666667]
mhb99    210201
[-0.14166667  0.125     ]
[0.125 0.25 ]
mhb99    210202
[0.00833333 0.1       ]
[-0.25        0.28333333]
mhb100    210301
[0.2   0.275]
[0.15833333 0.125     ]
mhb100    210302
[0.00833333 0.21666667]
[0.03333333 0.40833333]
mhb100    210303
[0.39166667 0.23333333]
[0.28333333 0.43333333]
mhb100    210305
[-0.1   

[-0.2         0.63076923]
mhb100    210306
[-0.09090909  0.16818182]
[-0.04545455  0.26363636]
mhb108    210505
[-0.11666667 -0.23333333]
[0. 0.]
mhb108    210508
[0.06923077 0.56923077]
[-0.43846154  0.53076923]
mhb108    210509
[0.         0.49166667]
[0.         0.34166667]
mhb108    210510
[-0.01666667  0.425     ]
[-0.11666667  0.5       ]
mhb124    221017
[-0.16818182 -0.08636364]
[-0.08636364  0.05454545]
mhb124    221018
[-0.21666667  0.01666667]
[0.05833333 0.35      ]
mhb124    221019
segment length longer than pulse, length difference:  1
segment length longer than pulse, length difference:  1
[0.30833333 0.16666667]
[0.04166667 0.25833333]
mhb124    221020
[0.375 0.375]
[0.29166667 0.25      ]
mhb124    221021
[0.06666667 0.15      ]
[-0.24166667  0.325     ]
################# rep:  12   ##############
mhb99    210128
[-0.01666667 -0.15      ]
[-0.29166667 -0.05833333]
mhb99    210129
[-0.03333333  0.475     ]
[0.35833333 0.4       ]
mhb99    210130
[0.00833333 0.46666667]


[-0.14166667  0.15833333]
[0.08333333 0.26666667]
mhb99    210202
[-0.1        -0.25833333]
[-0.225  0.225]
mhb100    210301
[0.38333333 0.21666667]
[0.15       0.09166667]
mhb100    210302
[0.04166667 0.05833333]
[0.06666667 0.38333333]
mhb100    210303
[0.19166667 0.025     ]
[0.3   0.425]
mhb100    210305
[-0.00769231  0.36153846]
[-0.15384615  0.60769231]
mhb100    210306
[0.09090909 0.09545455]
[-0.00909091  0.21818182]
mhb108    210505
[-0.11666667 -0.2       ]
[0. 0.]
mhb108    210508
[0.04615385 0.54615385]
[-0.14615385  0.33846154]
mhb108    210509
[0.         0.51666667]
[0.         0.41666667]
mhb108    210510
[-0.025       0.48333333]
[0.3   0.525]
mhb124    221017
[0.26818182 0.22272727]
[-0.13181818 -0.02272727]
mhb124    221018
[-0.16666667 -0.01666667]
[0.04166667 0.25833333]
mhb124    221019
segment length longer than pulse, length difference:  1
segment length longer than pulse, length difference:  1
[0.10833333 0.4       ]
[0.10833333 0.24166667]
mhb124    221020
[0.

In [5]:
#l_df.to_csv('results/LED_Classifier/all_cells/AL_tone_led_230602.csv')
# save results
al_df = pd.read_csv('results/tone_led_tone_active_230531.csv')

# Inference days

# RL and INF days - LED

In [4]:
mouse_ids = ['mhb99','mhb100','mhb108', 'mhb124']

rec_days = {'mhb99':['210204','210205','210208', '210209','210210','210211'],
            'mhb100':['210307', '210308', '210309', '210310','210312','210313','210314','210315','210316','210317'],
            'mhb108':['210511', '210512', '210513','210515','210516','210519','210520','210521'],
            'mhb124':['221028', '221029', '221030', '221102']}

correct_choices = ['incorrect']

In [5]:
#########################

In [6]:
correct_choice = 'incorrect'

res = {'mouse':[], 'day':[],'accs':[],'acc_type':[],'layer':[]}

for rep in range(20):
    print('############ rep', rep, '  ###############')
    for mouse_id in mouse_ids:

        for rr, rec_day in enumerate(rec_days[mouse_id]):

            print(mouse_id , '  ', rec_day)
            cue_pulse_names = ['ledsucrose','ledwater']
            sess_type = 'rl'
            cue_pops = get_layer_cuepop(mouse_id, rec_day, cue_pulse_names, sess_type, correct_trials= correct_choice)
            
            if cue_pops == None:
                print('empty cue_pop')
                continue

            ## sample same number of cells from each layer
            select_cells = np.min([cue_pops[0]['ledsucrose'].shape[1],cue_pops[1]['ledsucrose'].shape[1]])
            
            for layer, layer_name in enumerate(['L23', 'L56']):

                cue_pop = cue_pops[layer]
                
                if cue_pop['ledsucrose'].shape[1] != select_cells:

                    select_idx = random.sample(range(cue_pop['ledsucrose'].shape[1]),select_cells)
                    select_idx.sort()
                    for cue in cue_pop:
                        cue_pop[cue] = cue_pop[cue][:, select_idx]
                
                print(layer, '  ', cue_pop['ledsucrose'].shape)

                X, Y, Y_string = extract_XY(cue_pop)
 
                unique, counts = np.unique(Y, return_counts = True)
                if (len(unique)<2) or (min(counts)<2) or (X.shape[1] < 1):
                    do_clf=0
                    print('skipped : ', mouse_id, '  ', rec_day)
                else:
                    do_clf=1

                if do_clf:

                    cv = RepeatedStratifiedKFold(n_splits=np.min(counts),n_repeats=10)
                    scores = []
                    for i, (train_index, test_index) in enumerate(cv.split(X,Y)):
                        X_test, X_train, Y_test, Y_train  = X[test_index], X[train_index], Y[test_index], Y[train_index]
                    
                        # scale and transform
                        scaling = StandardScaler()
                        scaling.fit(X_train)
                        # Transform training and test
                        X_test_scaled = scaling.transform(X_test)
                        
                        model =  LogisticRegression(penalty='l1',solver='liblinear',random_state=0).fit(X_train,Y_train)
                        score1 = balanced_accuracy_score(Y_test, model.predict(X_test_scaled), adjusted=True)
                        scores += [score1]
                    res['accs'] += [np.mean(scores)]
                    res['mouse']+= [mouse_id]
                    res['day']+= [mouse_rec_days[mouse_id][rec_day]]
                    res['layer']+= [layer_name]
                    res['acc_type'] += ['led']

led_inf = pd.DataFrame(res)
inf_p = led_inf.groupby(['mouse', 'day', 'acc_type','layer']).agg({'accs':'mean'}).reset_index()

############ rep 0   ###############
mhb99    210204
0    (11, 2)
1    (11, 2)
mhb99    210205
0    (3, 30)
1    (3, 30)
mhb99    210208
0    (2, 19)
1    (2, 19)
mhb99    210209
0    (3, 19)
1    (3, 19)
mhb99    210210
0    (1, 24)
skipped :  mhb99    210210
1    (1, 24)
skipped :  mhb99    210210
mhb99    210211
0    (2, 29)
1    (2, 29)
mhb100    210307
0    (3, 16)
1    (3, 16)
mhb100    210308
0    (5, 27)
1    (5, 27)
mhb100    210309
0    (5, 27)
1    (5, 27)
mhb100    210310
0    (2, 20)
1    (2, 20)
mhb100    210312
0    (3, 19)
1    (3, 19)
mhb100    210313
0    (1, 21)
skipped :  mhb100    210313
1    (1, 21)
skipped :  mhb100    210313
mhb100    210314
0    (1, 19)
skipped :  mhb100    210314
1    (1, 19)
skipped :  mhb100    210314
mhb100    210315
0    (3, 24)
1    (3, 24)
mhb100    210316
0    (0, 26)
skipped :  mhb100    210316
1    (0, 26)
skipped :  mhb100    210316
mhb100    210317
0    (3, 33)
1    (3, 33)
mhb108    210511
0    (1, 11)
skipped :  mhb108    210511
1

0    (11, 2)
1    (11, 2)
mhb99    210205
0    (3, 30)
1    (3, 30)
mhb99    210208
0    (2, 19)
1    (2, 19)
mhb99    210209
0    (3, 19)
1    (3, 19)
mhb99    210210
0    (1, 24)
skipped :  mhb99    210210
1    (1, 24)
skipped :  mhb99    210210
mhb99    210211
0    (2, 29)
1    (2, 29)
mhb100    210307
0    (3, 16)
1    (3, 16)
mhb100    210308
0    (5, 27)
1    (5, 27)
mhb100    210309
0    (5, 27)
1    (5, 27)
mhb100    210310
0    (2, 20)
1    (2, 20)
mhb100    210312
0    (3, 19)
1    (3, 19)
mhb100    210313
0    (1, 21)
skipped :  mhb100    210313
1    (1, 21)
skipped :  mhb100    210313
mhb100    210314
0    (1, 19)
skipped :  mhb100    210314
1    (1, 19)
skipped :  mhb100    210314
mhb100    210315
0    (3, 24)
1    (3, 24)
mhb100    210316
0    (0, 26)
skipped :  mhb100    210316
1    (0, 26)
skipped :  mhb100    210316
mhb100    210317
0    (3, 33)
1    (3, 33)
mhb108    210511
0    (1, 11)
skipped :  mhb108    210511
1    (1, 11)
skipped :  mhb108    210511
mhb108    210

0    (11, 2)
1    (11, 2)
mhb99    210205
0    (3, 30)
1    (3, 30)
mhb99    210208
0    (2, 19)
1    (2, 19)
mhb99    210209
0    (3, 19)
1    (3, 19)
mhb99    210210
0    (1, 24)
skipped :  mhb99    210210
1    (1, 24)
skipped :  mhb99    210210
mhb99    210211
0    (2, 29)
1    (2, 29)
mhb100    210307
0    (3, 16)
1    (3, 16)
mhb100    210308
0    (5, 27)
1    (5, 27)
mhb100    210309
0    (5, 27)
1    (5, 27)
mhb100    210310
0    (2, 20)
1    (2, 20)
mhb100    210312
0    (3, 19)
1    (3, 19)
mhb100    210313
0    (1, 21)
skipped :  mhb100    210313
1    (1, 21)
skipped :  mhb100    210313
mhb100    210314
0    (1, 19)
skipped :  mhb100    210314
1    (1, 19)
skipped :  mhb100    210314
mhb100    210315
0    (3, 24)
1    (3, 24)
mhb100    210316
0    (0, 26)
skipped :  mhb100    210316
1    (0, 26)
skipped :  mhb100    210316
mhb100    210317
0    (3, 33)
1    (3, 33)
mhb108    210511
0    (1, 11)
skipped :  mhb108    210511
1    (1, 11)
skipped :  mhb108    210511
mhb108    210

0    (11, 2)
1    (11, 2)
mhb99    210205
0    (3, 30)
1    (3, 30)
mhb99    210208
0    (2, 19)
1    (2, 19)
mhb99    210209
0    (3, 19)
1    (3, 19)
mhb99    210210
0    (1, 24)
skipped :  mhb99    210210
1    (1, 24)
skipped :  mhb99    210210
mhb99    210211
0    (2, 29)
1    (2, 29)
mhb100    210307
0    (3, 16)
1    (3, 16)
mhb100    210308
0    (5, 27)
1    (5, 27)
mhb100    210309
0    (5, 27)
1    (5, 27)
mhb100    210310
0    (2, 20)
1    (2, 20)
mhb100    210312
0    (3, 19)
1    (3, 19)
mhb100    210313
0    (1, 21)
skipped :  mhb100    210313
1    (1, 21)
skipped :  mhb100    210313
mhb100    210314
0    (1, 19)
skipped :  mhb100    210314
1    (1, 19)
skipped :  mhb100    210314
mhb100    210315
0    (3, 24)
1    (3, 24)
mhb100    210316
0    (0, 26)
skipped :  mhb100    210316
1    (0, 26)
skipped :  mhb100    210316
mhb100    210317
0    (3, 33)
1    (3, 33)
mhb108    210511
0    (1, 11)
skipped :  mhb108    210511
1    (1, 11)
skipped :  mhb108    210511
mhb108    210

0    (11, 2)
1    (11, 2)
mhb99    210205
0    (3, 30)
1    (3, 30)
mhb99    210208
0    (2, 19)
1    (2, 19)
mhb99    210209
0    (3, 19)
1    (3, 19)
mhb99    210210
0    (1, 24)
skipped :  mhb99    210210
1    (1, 24)
skipped :  mhb99    210210
mhb99    210211
0    (2, 29)
1    (2, 29)
mhb100    210307
0    (3, 16)
1    (3, 16)
mhb100    210308
0    (5, 27)
1    (5, 27)
mhb100    210309
0    (5, 27)
1    (5, 27)
mhb100    210310
0    (2, 20)
1    (2, 20)
mhb100    210312
0    (3, 19)
1    (3, 19)
mhb100    210313
0    (1, 21)
skipped :  mhb100    210313
1    (1, 21)
skipped :  mhb100    210313
mhb100    210314
0    (1, 19)
skipped :  mhb100    210314
1    (1, 19)
skipped :  mhb100    210314
mhb100    210315
0    (3, 24)
1    (3, 24)
mhb100    210316
0    (0, 26)
skipped :  mhb100    210316
1    (0, 26)
skipped :  mhb100    210316
mhb100    210317
0    (3, 33)
1    (3, 33)
mhb108    210511
0    (1, 11)
skipped :  mhb108    210511
1    (1, 11)
skipped :  mhb108    210511
mhb108    210

In [8]:
#inf_p.to_csv('results/LED_Classifier/all_cells/INF_led_230604.csv')
inf_p.to_csv('results/LED_Classifier/all_cells/INF_led_incorrect_230609.csv')

# INF Days: tone

In [32]:
mouse_ids = ['mhb99','mhb100','mhb108', 'mhb124']

rec_days = {'mhb99':['210208', '210209','210210','210211'],
            'mhb100':['210310','210312','210313','210314','210315','210316','210317'],
            'mhb108':['210515','210516','210519','210520','210521'],
            'mhb124':['221028', '221029', '221030', '221102']}

correct_choices = ['correct']

In [33]:
correct_choice = 'correct'

res = {'mouse':[], 'day':[],'accs':[],'acc_type':[],'layer':[]}

for rep in range(20):
    print('############ rep', rep, '  ###############')
    for mouse_id in mouse_ids:

        for rr, rec_day in enumerate(rec_days[mouse_id]):

            print(mouse_id , '  ', rec_day)
            cue_pulse_names = ['ledsucrose','ledwater']
            sess_type = 'rl'
            cue_pops = get_layer_cuepop(mouse_id, rec_day, cue_pulse_names, sess_type, correct_trials= correct_choice)
            
            sess_type = 'tt'
            cue_pulse_names = ['toneSucrose','toneWater']
            cue_pop_tts = get_layer_cuepop(mouse_id, rec_day, cue_pulse_names, sess_type, correct_trials= correct_choice)
            cue_pop_tt_shifts = get_layer_cuepop(mouse_id, rec_day, cue_pulse_names, sess_type, shift = True, correct_trials= correct_choice)

            if cue_pops == None:
                print('empty cue_pop')
                continue

            ## sample same number of cells from each layer
            select_cells = np.min([cue_pops[0]['ledsucrose'].shape[1],cue_pops[1]['ledsucrose'].shape[1]])
            
            for layer, layer_name in enumerate(['L23', 'L56']):

                cue_pop = cue_pops[layer]
                cue_pop_tt = cue_pop_tts[layer]
                cue_pop_tt_shift = cue_pop_tt_shifts[layer]
                
                if cue_pop['ledsucrose'].shape[1] != select_cells:

                    select_idx = random.sample(range(cue_pop['ledsucrose'].shape[1]),select_cells)
                    select_idx.sort()
                    for cue in cue_pop:
                        cue_pop[cue] = cue_pop[cue][:, select_idx]
                    for cue in cue_pop_tt:
                        cue_pop_tt[cue] = cue_pop_tt[cue][:, select_idx]
                        cue_pop_tt_shift[cue] = cue_pop_tt_shift[cue][:, select_idx]
                
                print(layer, '  ', cue_pop['ledsucrose'].shape)

                X, Y, Y_string = extract_XY(cue_pop)
                X_t1, Y_t1, Y_string_t1 = extract_XY(cue_pop_tt)
                X_t2, Y_t2, Y_string_t2 = extract_XY(cue_pop_tt_shift)

                unique, counts = np.unique(Y, return_counts = True)
                uniquet, countst = np.unique(Y_t1, return_counts = True)
                if (len(unique)<2) or (min(counts)<2) or (X.shape[1] < 1) or (len(uniquet)<2) or (min(countst)<2):
                    do_clf=0
                    print('skipped : ', mouse_id, '  ', rec_day)
                else:
                    do_clf=1

                if do_clf:

                    cv = RepeatedStratifiedKFold(n_splits=np.min(counts),n_repeats=10)
                
                    # scale and transform
                    scaling = StandardScaler()
                    scaling.fit(X)

                    # Transform training and test
                    X_t1_scaled = scaling.transform(X_t1)
                    X_t2_scaled = scaling.transform(X_t2)

                    model =  LogisticRegression(penalty='l1',solver='liblinear',random_state=0).fit(X,Y)
                    score1 = balanced_accuracy_score(Y_t1, model.predict(X_t1_scaled), adjusted=True)
                    score2 = balanced_accuracy_score(Y_t2, model.predict(X_t2_scaled), adjusted=True)
                    res['accs'] += [score1, score2]
                    res['acc_type'] += ['t1', 't2']
                    res['mouse']+= [mouse_id]*2
                    res['day']+= [rr]*2
                    res['layer']+= [layer_name]*2

df_inf_tone = pd.DataFrame(res)
inf_tone = df_inf_tone.groupby(['mouse', 'day', 'acc_type','layer']).agg({'accs':'mean'}).reset_index()


############ rep 0   ###############
mhb99    210208
0    (4, 19)
skipped :  mhb99    210208
1    (4, 19)
skipped :  mhb99    210208
mhb99    210209
0    (7, 19)
1    (7, 19)
mhb99    210210
0    (7, 24)
skipped :  mhb99    210210
1    (7, 24)
skipped :  mhb99    210210
mhb99    210211
0    (4, 29)
1    (4, 29)
mhb100    210310
0    (5, 20)
1    (5, 20)
mhb100    210312
0    (4, 19)
1    (4, 19)
mhb100    210313
0    (5, 21)
1    (5, 21)
mhb100    210314
0    (7, 19)
skipped :  mhb100    210314
1    (7, 19)
skipped :  mhb100    210314
mhb100    210315
0    (7, 24)
1    (7, 24)
mhb100    210316
0    (6, 26)
skipped :  mhb100    210316
1    (6, 26)
skipped :  mhb100    210316
mhb100    210317
0    (6, 33)
1    (6, 33)
mhb108    210515
0    (6, 21)
1    (6, 21)
mhb108    210516
0    (5, 21)
skipped :  mhb108    210516
1    (5, 21)
skipped :  mhb108    210516
mhb108    210519
0    (6, 23)
1    (6, 23)
mhb108    210520
0    (3, 20)
skipped :  mhb108    210520
1    (3, 20)
skipped :  mhb108 

0    (4, 29)
1    (4, 29)
mhb100    210310
0    (5, 20)
1    (5, 20)
mhb100    210312
0    (4, 19)
1    (4, 19)
mhb100    210313
0    (5, 21)
1    (5, 21)
mhb100    210314
0    (7, 19)
skipped :  mhb100    210314
1    (7, 19)
skipped :  mhb100    210314
mhb100    210315
0    (7, 24)
1    (7, 24)
mhb100    210316
0    (6, 26)
skipped :  mhb100    210316
1    (6, 26)
skipped :  mhb100    210316
mhb100    210317
0    (6, 33)
1    (6, 33)
mhb108    210515
0    (6, 21)
1    (6, 21)
mhb108    210516
0    (5, 21)
skipped :  mhb108    210516
1    (5, 21)
skipped :  mhb108    210516
mhb108    210519
0    (6, 23)
1    (6, 23)
mhb108    210520
0    (3, 20)
skipped :  mhb108    210520
1    (3, 20)
skipped :  mhb108    210520
mhb108    210521
0    (3, 23)
skipped :  mhb108    210521
1    (3, 23)
skipped :  mhb108    210521
mhb124    221028
0    (5, 10)
1    (5, 10)
mhb124    221029
0    (5, 14)
1    (5, 14)
mhb124    221030
0    (5, 6)
1    (5, 6)
mhb124    221102
0    (5, 7)
skipped :  mhb124    2

0    (7, 24)
1    (7, 24)
mhb100    210316
0    (6, 26)
skipped :  mhb100    210316
1    (6, 26)
skipped :  mhb100    210316
mhb100    210317
0    (6, 33)
1    (6, 33)
mhb108    210515
0    (6, 21)
1    (6, 21)
mhb108    210516
0    (5, 21)
skipped :  mhb108    210516
1    (5, 21)
skipped :  mhb108    210516
mhb108    210519
0    (6, 23)
1    (6, 23)
mhb108    210520
0    (3, 20)
skipped :  mhb108    210520
1    (3, 20)
skipped :  mhb108    210520
mhb108    210521
0    (3, 23)
skipped :  mhb108    210521
1    (3, 23)
skipped :  mhb108    210521
mhb124    221028
0    (5, 10)
1    (5, 10)
mhb124    221029
0    (5, 14)
1    (5, 14)
mhb124    221030
0    (5, 6)
1    (5, 6)
mhb124    221102
0    (5, 7)
skipped :  mhb124    221102
1    (5, 7)
skipped :  mhb124    221102
############ rep 13   ###############
mhb99    210208
0    (4, 19)
skipped :  mhb99    210208
1    (4, 19)
skipped :  mhb99    210208
mhb99    210209
0    (7, 19)
1    (7, 19)
mhb99    210210
0    (7, 24)
skipped :  mhb99    

0    (5, 21)
skipped :  mhb108    210516
1    (5, 21)
skipped :  mhb108    210516
mhb108    210519
0    (6, 23)
1    (6, 23)
mhb108    210520
0    (3, 20)
skipped :  mhb108    210520
1    (3, 20)
skipped :  mhb108    210520
mhb108    210521
0    (3, 23)
skipped :  mhb108    210521
1    (3, 23)
skipped :  mhb108    210521
mhb124    221028
0    (5, 10)
1    (5, 10)
mhb124    221029
0    (5, 14)
1    (5, 14)
mhb124    221030
0    (5, 6)
1    (5, 6)
mhb124    221102
0    (5, 7)
skipped :  mhb124    221102
1    (5, 7)
skipped :  mhb124    221102
############ rep 19   ###############
mhb99    210208
0    (4, 19)
skipped :  mhb99    210208
1    (4, 19)
skipped :  mhb99    210208
mhb99    210209
0    (7, 19)
1    (7, 19)
mhb99    210210
0    (7, 24)
skipped :  mhb99    210210
1    (7, 24)
skipped :  mhb99    210210
mhb99    210211
0    (4, 29)
1    (4, 29)
mhb100    210310
0    (5, 20)
1    (5, 20)
mhb100    210312
0    (4, 19)
1    (4, 19)
mhb100    210313
0    (5, 21)
1    (5, 21)
mhb100    

In [35]:
inf_tone.to_csv('results/LED_Classifier/all_cells/INF_tone_230605.csv')
